In [85]:
import pandas as pd 
import numpy as np

In [86]:
dataset = pd.read_csv('ratings.csv')
dataset.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [87]:
movieTitles = pd.read_csv('movies.csv')
movieTitles.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [88]:
dataset = pd.merge(dataset, movieTitles, on='movieId')
dataset.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [89]:
movieMatrix = dataset.pivot_table(index='userId', columns='movieId', values='rating')
movieMatrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
userIdMap = {}
tempInd = 0
for i in dataset['userId'].unique():
    userIdMap[i] = tempInd
    tempInd += 1

In [91]:
itemIdMap = {}
tempInd = 0
for i in dataset['movieId'].unique():
    itemIdMap[i] = tempInd
    tempInd += 1

In [92]:
train = pd.read_csv('train.csv')

In [93]:
movieMatrix = np.array(movieMatrix)

movieMatrix[:,:] = np.nan
for i in train.index:
    movieMatrix[userIdMap[train['userId'][i]], itemIdMap[train['movieId'][i]]] = train['rating'][i]
np.count_nonzero(~np.isnan(movieMatrix))

70829

In [94]:
test = pd.read_csv('test.csv')

testMatrix = np.empty((610, 9724))
testMatrix[:,:] = 0
for i in test.index:
    testMatrix[userIdMap[test['userId'][i]], itemIdMap[test['movieId'][i]]] = test['rating'][i]
testMatrix

array([[0., 4., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [95]:
numUsers, numItems = movieMatrix.shape

In [96]:
where_are_NaNs = np.isnan(movieMatrix)
where_are_NaNs = np.array(where_are_NaNs)
where_are_NaNs

array([[False,  True, False, ...,  True,  True,  True],
       [False,  True,  True, ...,  True,  True,  True],
       [False,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [108]:
R = movieMatrix
R[where_are_NaNs] = 0
R

array([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
       [4. , 0. , 0. , ..., 0. , 0. , 0. ],
       [4.5, 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [98]:
K = 400

In [99]:
regularizationFactor = 100

In [100]:
alpha = 0.000001

In [101]:
epsilon = 0.0000000001

In [102]:
U = np.random.rand(numUsers, K)
U

array([[0.13860188, 0.16212164, 0.02385412, ..., 0.93880036, 0.40153586,
        0.89293369],
       [0.56700807, 0.07620385, 0.80001903, ..., 0.36624759, 0.15266617,
        0.5916097 ],
       [0.25981377, 0.5527702 , 0.57110341, ..., 0.28332355, 0.52268015,
        0.00333677],
       ...,
       [0.85698155, 0.54278317, 0.93721406, ..., 0.44717013, 0.00289998,
        0.5031943 ],
       [0.19399163, 0.12733484, 0.64333624, ..., 0.82114978, 0.01866688,
        0.19462725],
       [0.41039428, 0.11661751, 0.24745475, ..., 0.48217337, 0.32389279,
        0.6126917 ]])

In [103]:
V = np.random.rand(numItems, K)
V

array([[0.79863509, 0.05974697, 0.72272082, ..., 0.88199448, 0.06419121,
        0.21421875],
       [0.34189694, 0.08627256, 0.1101762 , ..., 0.30708755, 0.94309627,
        0.09807966],
       [0.85812072, 0.7064071 , 0.13709011, ..., 0.83549114, 0.10785865,
        0.33867229],
       ...,
       [0.04354181, 0.43030771, 0.2619779 , ..., 0.83942768, 0.59199849,
        0.91883037],
       [0.16763601, 0.12639172, 0.91942107, ..., 0.46124984, 0.87724965,
        0.25167648],
       [0.22354676, 0.3787688 , 0.07639105, ..., 0.47892272, 0.5574855 ,
        0.27462604]])

In [104]:
# for i in range(50000):
#     # Calculating norm
#     error = R - np.matmul(U, V.transpose())
#     error[where_are_NaNs] = 0
#     print("Iteration: ", i, " ---> Frobenius norm: ", np.linalg.norm(error))
    
#     RV = np.matmul(R, V)
#     UVt = np.matmul(U, V.T)
#     UVtV = np.matmul(UVt, V)
    
#     RtU = np.matmul(R.T, U)
#     VUt = np.matmul(V, U.T)
#     VUtU = np.matmul(VUt, U)
    
#     #Batch Update
#     U_updated = np.empty((numUsers, K))
#     V_updated = np.empty((numItems, K))
#     for j in range(0, numUsers):
#         for k in range(0, K):
#             U_updated[j, k] = max(0, ( (RV[j, k] - regularizationFactor*U[j, k]) / (UVtV[j, k] + epsilon)) * U[j, k] )
    
#     for j in range(0, numItems):
#         for k in range(0, K):
#             V_updated[j, k] = max(0, ( (RtU[j, k] - regularizationFactor*V[j, k]) / (VUtU[j, k] + epsilon)) * V[j, k] )
    
#     U = U_updated
#     V = V_updated
    

In [105]:
from sklearn.decomposition import NMF

In [106]:
model = NMF(n_components=K, init='random', random_state=0)

In [115]:
W = model.fit(R)

In [121]:
W.transform(R)

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.11051603,
        0.        ]])

In [116]:
H = model.components_

In [120]:
W.shape

AttributeError: 'NMF' object has no attribute 'shape'

In [117]:
filledMatrix = np.matmul(W, H)
filledMatrix

ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [114]:
errorBatch = testMatrix-filledMatrix
errorBatch[where_are_NaNs] = 0
np.linalg.norm(errorBatch)

940.4430136937881